In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torch.utils.data as Data
from torch.nn import init
import torch.optim as optim

from importlib import reload
import pytorch as pts

In [29]:
torch.manual_seed(1)
print(torch.__version__)
torch.set_default_tensor_type("torch.FloatTensor")

1.10.2


# Generate Data

In [30]:
num_inputs = 2
num_examples = 1000
true_w = np.array([[2, -3.4]])
true_b = 4.2
features = np.random.normal(0, 1, (num_examples, num_inputs))
labels = np.dot(features, true_w.T) + true_b 
labels += np.random.normal(0, 0.01, (num_examples, 1))
lables = labels.reshape(num_examples)

# Read Data

In [32]:
features = torch.tensor(features, dtype=torch.float)
labels = torch.tensor(lables, dtype=torch.float)
data_iter = pts.pack_data(features, labels)

# Define Model

In [33]:
# 写法一
net = nn.Sequential(
    nn.Linear(num_inputs, 1)
    # 此处还可以传入其他层
    )

# 写法二
net = nn.Sequential()
net.add_module('linear', nn.Linear(num_inputs, 1))
# net.add_module ......

# 写法三
from collections import OrderedDict
net = nn.Sequential(OrderedDict([
          ('linear', nn.Linear(num_inputs, 1))
          # ......
        ]))

## Loss Function

In [36]:
loss = nn.MSELoss()

## Optimization Algorithm

In [37]:
optimizer = optim.SGD(net.parameters(), lr=0.03)

# 为不同子网络设置不同的学习率
# optimizer =optim.SGD([
#                 # 如果对某个参数不指定学习率，就使用最外层的默认学习率
#                 {'params': net.subnet1.parameters()}, # lr=0.03
#                 {'params': net.subnet2.parameters(), 'lr': 0.01}
#             ], lr=0.03)
# # 调整学习率
# for param_group in optimizer.param_groups:
#     param_group['lr'] *= 0.1 # 学习率为之前的0.1倍
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.03
    momentum: 0
    nesterov: False
    weight_decay: 0
)


## Fit Model

In [38]:
## 使用缺省的训练模型
#pts.fit(net, data_iter)  ## 使用缺省的损失函数与优化器
pts.fit(net, data_iter, loss, optimizer, 4)

epoch 1, loss: 0.000103
epoch 2, loss: 0.000100
epoch 3, loss: 0.000055
epoch 4, loss: 0.000073


## The Results

In [39]:
dense = net.linear
print(true_w, "\t", dense.weight.data)
print(true_b, "\t", dense.bias.data)

[[ 2.  -3.4]] 	 tensor([[ 2.0001, -3.4002]])
4.2 	 tensor([4.1998])
